## <font color='darkblue'>GE Imp (1)</font>
([course link](https://www.udemy.com/course/ai-and-combinatorial-optimization-with-meta-heuristics/learn/lecture/30477164?start=0#overview))

In [44]:
from random import uniform
from numpy.random import randint
from typing import List

SOLUTION_SEQUENCE = list(range(10))
TOURNAMENT_SIZE = 20
MAX_FITNESS = 10
CHROMOSOME_LENGTH = 10

# Chromosome
class Individual:
  """Representation of the problem."""
  
  def __init__(self):
    self.genes = [
      randint(CHROMOSOME_LENGTH) for _ in range(CHROMOSOME_LENGTH)
    ]
    
  def get_fitness(self):
    """Calculates the fitness values of these chromosomes"""
    fitness = sum(
      map(
        lambda i: self.genes[i] == SOLUTION_SEQUENCE[i],
        range(CHROMOSOME_LENGTH),
      ))        
    return fitness
  
  def __repr__(self):
    return ''.join(map(str, self.genes))

## <font color='darkblue'>GE Imp (2)</font>
([course link](https://www.udemy.com/course/ai-and-combinatorial-optimization-with-meta-heuristics/learn/lecture/30477166#overview))

In [2]:
values = [
  (3, 'three'),
  (6, 'six'),
  (1, 'one'),
  (5, 'five')
]
index_min = max(values, key=lambda t: t[0])[1]
print(index_min)

six


In [72]:
class Population:
  
  def __init__(self, population_size):
    self._population_size = population_size
    self._individuals = (
      [Individual() for _ in range(self._population_size)])

  def iter_individuals(self):
    for i in self._individuals:
      yield i
    
  def get_fittest(self):
    return max(
      map(lambda i: (i, i.get_fitness()), self._individuals),
      key=lambda t: t[1])[0]
  
  def get_fittest_elitism(self, n: int) -> List[Individual]:
    self._individuals.sort(
      key=lambda ind: ind.get_fitness(), reverse=True)
    return self._individuals[:n]
  
  def get_size(self):
    return self._population_size
  
  def get_individual(self, index: int):
    return self._individuals[index]
  
  def save_individual(self, index: int, individual: Individual):
    self._individuals[index] = individual

## <font color='darkblue'>GE Imp (3)</font>
([course link](https://www.udemy.com/course/ai-and-combinatorial-optimization-with-meta-heuristics/learn/lecture/30477168#overview))

In [68]:
class GeneticAlgorithm:
  def __init__(
    self, population_size: int=100, crossover_rate: float=0.65,
    mutation_rate: float=0.1):
    self._population_size = population_size
    self._crossover_rate = crossover_rate
    self._mutation_rate = mutation_rate
    
  def run(self):
    pop = Population(self._population_size)
    generation_counter = 0
    
    while pop.get_fittest().get_fitness() != MAX_FITNESS:
      generation_counter += 1
      print(
        f'Generation #{generation_counter} - '
        f'fittest is : {pop.get_fittest()} '
        f'with fitness value {pop.get_fittest().get_fitness()}')
      
      pop = self.evolve_population(pop)
      
    print('Solution found!')
    print(pop.get_fittest())
    
  def crossover(self, indv1, indv2):
    cross_individual = Individual()
    start = randint(CHROMOSOME_LENGTH)
    end = randint(start, CHROMOSOME_LENGTH)
    
    cross_individual.genes = (
      indv1.genes[:start] + indv2.genes[start:end] + indv1.genes[end:])
    return cross_individual
    
  def evolve_population(self, population: Population) -> Population:
    """Evolves given population"""
    next_population = Population(self._population_size)
    
    # Crossover (Tournament selection)
    for i in range(next_population.get_size()):
      first = self.random_selection(population)
      second = self.random_selection(population)
      next_population.save_individual(i, self.crossover(first, second))
      
    # Mutation
    for individual in next_population.iter_individuals():
      self.mutate(individual)
      
    return next_population
  
  def mutate(self, individual):
    for index in range(CHROMOSOME_LENGTH):
      if uniform(0, 1) < self._mutation_rate:
        individual.genes[index] = randint(CHROMOSOME_LENGTH)
  
  def random_selection(self, actual_population) -> Individual:
    """Does tournament selection."""
    new_population = Population(TOURNAMENT_SIZE)
    
    for i in range(TOURNAMENT_SIZE):
      random_index = randint(actual_population.get_size())
      new_population.save_individual(
        i, actual_population.get_individual(random_index))
      
    return new_population.get_fittest()

## <font color='darkblue'>GE Imp (4)</font>
([course link](https://www.udemy.com/course/ai-and-combinatorial-optimization-with-meta-heuristics/learn/lecture/30552652#overview))

In [35]:
algorithm = GeneticAlgorithm(population_size=50)

In [36]:
algorithm.run()

Generation #1 - fittest is : 7229559682 with fitness value 3
Generation #2 - fittest is : 7113045782 with fitness value 4
Generation #3 - fittest is : 3128040789 with fitness value 5
Generation #4 - fittest is : 3128050789 with fitness value 6
Generation #5 - fittest is : 3128050789 with fitness value 6
Generation #6 - fittest is : 6123050789 with fitness value 7
Generation #7 - fittest is : 6123050789 with fitness value 7
Generation #8 - fittest is : 6123450789 with fitness value 8
Generation #9 - fittest is : 6123450789 with fitness value 8
Generation #10 - fittest is : 6123450789 with fitness value 8
Generation #11 - fittest is : 6123456789 with fitness value 9
Generation #12 - fittest is : 6123456789 with fitness value 9
Generation #13 - fittest is : 6123456789 with fitness value 9
Generation #14 - fittest is : 6123456789 with fitness value 9
Generation #15 - fittest is : 3123456789 with fitness value 9
Generation #16 - fittest is : 7123456789 with fitness value 9
Solution found!
0

## <font color='darkblue'>GE Imp (Elitism)</font>
([course link](https://www.udemy.com/course/ai-and-combinatorial-optimization-with-meta-heuristics/learn/lecture/30552252#overview))

In [76]:
class GeneticAlgorithmWithElitism:
  def __init__(
    self, population_size: int=100, crossover_rate: float=0.65,
    mutation_rate: float=0.1, elitism_param: int=5):
    self._population_size = population_size
    self._crossover_rate = crossover_rate
    self._mutation_rate = mutation_rate
    self._elitism_param = elitism_param
    
  def run(self):
    pop = Population(self._population_size)
    generation_counter = 0
    
    while pop.get_fittest().get_fitness() != MAX_FITNESS:
      generation_counter += 1
      print(
        f'Generation #{generation_counter} - '
        f'fittest is : {pop.get_fittest()} '
        f'with fitness value {pop.get_fittest().get_fitness()}')
      
      pop = self.evolve_population(pop)
      
    print('Solution found!')
    print(pop.get_fittest())
    
  def crossover(self, indv1, indv2):
    cross_individual = Individual()
    start = randint(CHROMOSOME_LENGTH)
    end = randint(start, CHROMOSOME_LENGTH)
    
    cross_individual.genes = (
      indv1.genes[:start] + indv2.genes[start:end] + indv1.genes[end:])
    return cross_individual
    
  def evolve_population(self, population: Population) -> Population:
    """Evolves given population"""
    next_population = Population(self._population_size)
    next_population._individuals.extend(
      population.get_fittest_elitism(self._elitism_param))
    
    # Crossover (Tournament selection)
    for i in range(self._elitism_param, next_population.get_size()):
      first = self.random_selection(population)
      second = self.random_selection(population)
      next_population.save_individual(
        i, self.crossover(first, second))
      
    # Mutation
    for individual in next_population.iter_individuals():
      self.mutate(individual)
      
    return next_population
  
  def mutate(self, individual):
    for index in range(CHROMOSOME_LENGTH):
      if uniform(0, 1) < self._mutation_rate:
        individual.genes[index] = randint(CHROMOSOME_LENGTH)
  
  def random_selection(self, actual_population) -> Individual:
    """Does tournament selection."""
    new_population = Population(TOURNAMENT_SIZE)
    
    for i in range(TOURNAMENT_SIZE):
      random_index = randint(actual_population.get_size())
      new_population.save_individual(
        i, actual_population.get_individual(random_index))
      
    return new_population.get_fittest()

In [77]:
algorithm = GeneticAlgorithmWithElitism(population_size=50)

In [78]:
algorithm.run()

Generation #1 - fittest is : 5658352689 with fitness value 3
Generation #2 - fittest is : 0621476446 with fitness value 4
Generation #3 - fittest is : 5620458689 with fitness value 5
Generation #4 - fittest is : 9025456889 with fitness value 6
Generation #5 - fittest is : 5620456689 with fitness value 6
Generation #6 - fittest is : 0220456889 with fitness value 7
Generation #7 - fittest is : 9823456889 with fitness value 7
Generation #8 - fittest is : 9123456789 with fitness value 9
Solution found!
0123456789
